In [ ]:
import os, shutil
import pandas as pd
import numpy as np

path = os.getcwd()

if 'Young_Research' in path:
    initialpath = 'C:\\Users\\Young_Research\\Documents\\Rutgers\\Research\\2020_PHY_MSc\\data\\initial_conditions\\jul2020'
    pathff   = "C:\\Users\\Young_Research\\Documents\\Rutgers\\Research\\2020_PHY_MSc\\data\\forcefields"
else:
    pathff   = "/home/rty10/Documents/Experiments/Optimizations/Opt_ForceFields"

In [ ]:
incons      = ['pc14','pc15','or15']
inseqs      = ['col05','col06','col07','col08','col09']

def load_intrinsic_state(forcefield):
    if '-crt' in forcefield:
        ff = forcefield.replace('-crt','')
    elif '-frt' in forcefield:
        ff = forcefield.replace('-frt','')
    else:
        ff = forcefield
    
    infile = open(pathff+'/RestStateParameters/StepParameters_'+ff+'.txt', 'r')
    indata = infile.readlines()
    indata = [i.replace("={"," ").replace(", "," ").replace("}","").rstrip('\n').split() for i in indata]
    header = ['dimer','Tilt','Roll','Twist','Shift','Slide','Rise']
    for i in range(0, len(indata)):
        for j, x in enumerate(indata[i]):
            try:
                indata[i][j] = float(x)
            except ValueError:
                pass
    df1 = pd.DataFrame.from_records(indata, columns=header)
    df1['Bend'] = np.sqrt(df1.Tilt**2 + df1.Roll**2)
    df1 = df1.set_index('dimer')
    del indata
    return df1

In [ ]:
if not os.path.exists('opt_pars-diff'):
    os.mkdir('opt_pars-diff')
    
for filename in os.listdir(path+'/opt_par'):
    name, ext = filename.split('.')
    seq = name.split('_')[0]
    incon = name.split('_')[1]
    ff  = name.split('_')[2]

    rsdf = load_intrinsic_state(ff)

    initialdf = pd.read_csv(initialpath+'/in_circ_par/'+seq+'_'+incon+'.par', sep='\s+', skiprows=2)
    initialdf['Bend'] = np.sqrt( initialdf['Roll']**2 + initialdf['Tilt']**2 )
    initialdf = initialdf.drop([0], axis=0).reset_index()

    datadf = pd.read_csv(path+'/opt_par/'+name+'.par', sep='\s+', skiprows=2)
    datadf['Bend'] = np.sqrt( datadf['Roll']**2 + datadf['Tilt']**2 )
    for i in range(1, len(datadf)):
        datadf.at[i, 'dimer'] = datadf.at[i-1, '#'][0]+datadf.at[i, '#'][0]
    datadf = datadf.drop([0], axis=0).reset_index()

    diffdf = pd.DataFrame()
    for i in range(0, len(datadf)):
        diffdf.at[i, 'dimer']     = datadf.at[i, 'dimer']

        diffdf.at[i, 'excess_tilt']  = datadf.at[i, 'Tilt']  - rsdf.at[datadf.at[i, 'dimer'], 'Tilt']
        diffdf.at[i, 'excess_roll']  = datadf.at[i, 'Roll']  - rsdf.at[datadf.at[i, 'dimer'], 'Roll']
        diffdf.at[i, 'excess_bend']  = datadf.at[i, 'Bend']  - rsdf.at[datadf.at[i, 'dimer'], 'Bend']
        diffdf.at[i, 'excess_twist'] = datadf.at[i, 'Twist'] - rsdf.at[datadf.at[i, 'dimer'], 'Twist']

    diffdf['delta_tilt']  = datadf['Tilt']  - initialdf['Tilt']
    diffdf['delta_roll']  = datadf['Roll']  - initialdf['Roll']
    diffdf['delta_bend']  = datadf['Bend']  - initialdf['Bend']
    diffdf['delta_twist'] = datadf['Twist'] - initialdf['Twist']

    diffdf['shift_tilt'] = datadf['Tilt']  - datadf['Tilt'].mean()
    diffdf['shift_roll'] = datadf['Roll']  - datadf['Roll'].mean()
    diffdf['shift_bend'] = datadf['Bend']  - datadf['Bend'].mean()
    diffdf['shift_twist']= datadf['Twist'] - datadf['Twist'].mean()

    diffdf.to_csv(name+'_bps-diffdata.txt', index=False)

    shutil.move(name+'_bps-diffdata.txt', 'opt_pars-diff')

    del datadf, rsdf, diffdf
    del initialdf      
